# Shift Scheduling using Python-MIP


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

from mip import *


np.random.seed(0)

## Reading data


In [2]:
staff_df = pd.read_excel("./staff_data.xlsx", sheet_name="Staff Info")
vacation_requests_df = pd.read_excel("staff_data.xlsx", sheet_name="Desired day off")

display(staff_df, vacation_requests_df)

,staff_name,level,fulltime
0,staff_a,5,1
1,staff_b,4,1
2,staff_c,4,1
3,staff_d,3,0
4,staff_e,2,0


,staff_name,day_off
0,staff_a,2024-09-03
1,staff_a,2024-08-30
2,staff_c,2024-09-07


In [3]:
def get_closed_days(start_date, end_date):
    closed = []
    current_date = start_date
    while current_date <= end_date:
        if current_date.weekday() == 3:  # Monday is 0 and Sunday is 6
            closed.append(current_date)
        current_date += timedelta(days=1)

    # closed.append(datetime(2024, 9, 16)) # if you need to add a temporary closed day

    return [day.date() for day in closed]


# -------------------------------------------
start_date = datetime(2024, 8, 26)
end_date = start_date + timedelta(weeks=2) - timedelta(days=1)
# -------------------------------------------
all_days = {i: (start_date + timedelta(days=i)).date() for i in range((end_date - start_date).days + 1)}
closed_days = get_closed_days(start_date, end_date)
work_days = [day for day in all_days if day not in closed_days]
print(f"   coled day in this month ({len(closed_days)}):  {closed_days}")
print(f"   Opening days in this month:  {len(work_days)}, total days: {len(all_days)}")

day_off = {i: 1 if all_days[i] in closed_days else 0 for i in range(len(all_days))}

print((all_days))

   coled day in this month (2):  [datetime.date(2024, 8, 29), datetime.date(2024, 9, 5)]
   Opening days in this month:  14, total days: 14
{0: datetime.date(2024, 8, 26), 1: datetime.date(2024, 8, 27), 2: datetime.date(2024, 8, 28), 3: datetime.date(2024, 8, 29), 4: datetime.date(2024, 8, 30), 5: datetime.date(2024, 8, 31), 6: datetime.date(2024, 9, 1), 7: datetime.date(2024, 9, 2), 8: datetime.date(2024, 9, 3), 9: datetime.date(2024, 9, 4), 10: datetime.date(2024, 9, 5), 11: datetime.date(2024, 9, 6), 12: datetime.date(2024, 9, 7), 13: datetime.date(2024, 9, 8)}


In [4]:
S_vacation_request = {}
for row in vacation_requests_df.itertuples():
    staff = row.staff_name
    leave_date = pd.to_datetime(row.day_off).date()

    if staff not in S_vacation_request:
        S_vacation_request[staff] = []
    S_vacation_request[staff].append(leave_date)

display(S_vacation_request)

{'staff_a': [datetime.date(2024, 9, 3), datetime.date(2024, 8, 30)],
 'staff_c': [datetime.date(2024, 9, 7)]}

## Only one shift type


In [5]:
# -------------------------------------------
# Prepare data for Only one shift type
# -------------------------------------------
STAFFS = staff_df["staff_name"].unique()
WORKING_DAYS = {day for day in range(len(all_days))}

WORKING_DAYS_EXCLUDING_CLOSED = [d for d in WORKING_DAYS if day_off[d] != 1]
STAFFS_WORKING_DAYS = [(staff, day) for staff in STAFFS for day in WORKING_DAYS]
S_FULLTIME_OR_NOT = {row.staff_name: 1 if row.fulltime == 1 else 0 for row in staff_df.itertuples()}
S_LEVEL = {row.staff_name: row.level for row in staff_df.itertuples()}

In [6]:
model = Model(sense=MINIMIZE, solver_name=CBC)
x = {(s, d): model.add_var(var_type=BINARY) for s in STAFFS for d in WORKING_DAYS}


# Two staff members are required for each workday (Need 2 people for each day)
for d in WORKING_DAYS_EXCLUDING_CLOSED:
    model += xsum(x[s, d] for s in STAFFS) == 2
# Each staff work no more than 5 days per week
weeks = [list(WORKING_DAYS)[i : i + 7] for i in range(0, len(WORKING_DAYS), 7)]
for s in STAFFS:
    for week in weeks:
        model += xsum(x[s, d] for d in week) <= 5
# There be at least one full-time employee on each workday
for d in WORKING_DAYS_EXCLUDING_CLOSED:
    model += xsum(x[s, d] * S_FULLTIME_OR_NOT[s] for s in STAFFS) >= 1

# The difference in the number of working days between full-time employees is within 1 day
fulltime_staffs = [s for s in STAFFS if S_FULLTIME_OR_NOT[s] == 1]
for i in range(len(fulltime_staffs)):
    for j in range(i + 1, len(fulltime_staffs)):
        staff_i = fulltime_staffs[i]
        staff_j = fulltime_staffs[j]
        model += xsum(x[staff_i, d] for d in WORKING_DAYS_EXCLUDING_CLOSED) - xsum(x[staff_j, d] for d in WORKING_DAYS_EXCLUDING_CLOSED) <= 1
        model += xsum(x[staff_j, d] for d in WORKING_DAYS_EXCLUDING_CLOSED) - xsum(x[staff_i, d] for d in WORKING_DAYS_EXCLUDING_CLOSED) <= 1
# The difference in the number of working days between part-time employees is within 1 day
parttime_staffs = [s for s in STAFFS if S_FULLTIME_OR_NOT[s] == 0]
for i in range(len(parttime_staffs)):
    for j in range(i + 1, len(parttime_staffs)):
        staff_i = parttime_staffs[i]
        staff_j = parttime_staffs[j]
        model += xsum(x[staff_i, d] for d in WORKING_DAYS_EXCLUDING_CLOSED) - xsum(x[staff_j, d] for d in WORKING_DAYS_EXCLUDING_CLOSED) <= 1
        model += xsum(x[staff_j, d] for d in WORKING_DAYS_EXCLUDING_CLOSED) - xsum(x[staff_i, d] for d in WORKING_DAYS_EXCLUDING_CLOSED) <= 1
# Each staff member work at least one shift.
for s in STAFFS:
    model += xsum(x[s, d] for d in WORKING_DAYS_EXCLUDING_CLOSED) >= 1

# There is not much difference in the level of staff working on each date.
min_value = min(S_LEVEL.values())
max_value = max(S_LEVEL.values())
for d in WORKING_DAYS_EXCLUDING_CLOSED:
    level_sum = xsum(x[s, d] * S_LEVEL[s] for s in STAFFS)
    model += level_sum >= min_value + max_value  # Total Minimum Level
    model += level_sum <= max_value + max_value  # Total Maximum Level

# Constrain day-off requests
display(S_vacation_request)
for staff, leave_dates in S_vacation_request.items():
    for leave_date in leave_dates:
        if leave_date in all_days.values():
            day = [d for d in WORKING_DAYS if all_days[d] == leave_date][0]
            model += x[staff, day] == 0

model.optimize()
print(f"status: {model.status}")

{'staff_a': [datetime.date(2024, 9, 3), datetime.date(2024, 8, 30)],
 'staff_c': [datetime.date(2024, 9, 7)]}

Welcome to the CBC MILP Solver 
Version: Trunk
Build Date: Oct 28 2021 

Starting solution of the Linear programming relaxation problem using Primal Simplex

Clp0024I Matrix will be packed to eliminate 24 small elements
Coin0506I Presolve 68 (-6) rows, 57 (-13) columns and 483 (-58) elements
Clp1000I sum of infeasibilities 3.03291e-12 - average 4.46016e-14, 0 fixed columns
Coin0506I Presolve 68 (0) rows, 57 (0) columns and 483 (0) elements
Clp0029I End of values pass after 57 iterations
Clp0000I Optimal - objective value 0
Clp0000I Optimal - objective value 0
Clp0000I Optimal - objective value 0
Coin0511I After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 0 - 0 iterations time 0.012, Presolve 0.00, Idiot 0.00

Starting MIP optimization
Cgl0002I 3 variables fixed
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 0 substitutions
Cgl0004I processed model has 56 rows, 57 columns (57 integer (57 of which binary)) and 428 elements
Coi

#### output result


In [7]:
data = [[] for _ in range(len(STAFFS))]
for i, staff in enumerate(STAFFS):
    for d in WORKING_DAYS:
        if x[staff, d].x >= 0.99:
            data[i].append("◯")
        else:
            data[i].append("×")
df = pd.DataFrame(data, index=STAFFS, columns=[all_days[d] for d in WORKING_DAYS])
display(df)

,2024-08-26,2024-08-27,2024-08-28,2024-08-29,2024-08-30,2024-08-31,2024-09-01,2024-09-02,2024-09-03,2024-09-04,2024-09-05,2024-09-06,2024-09-07,2024-09-08
staff_a,◯,×,◯,×,×,◯,◯,×,×,×,×,◯,◯,◯
staff_b,◯,◯,×,×,◯,×,×,×,◯,◯,×,◯,◯,×
staff_c,×,◯,◯,×,◯,×,×,◯,◯,◯,×,×,×,◯
staff_d,×,×,×,×,×,×,◯,◯,×,×,×,×,×,×
staff_e,×,×,×,×,×,◯,×,×,×,×,×,×,×,×


## 2 shifts type


In [8]:
# -------------------------------------------
# Prepare data for 2 shifts type
# -------------------------------------------
STAFFS = staff_df["staff_name"].unique()
WORKING_DAYS = {day for day in range(len(all_days))}
SHIFT_TYPE = ["M", "E"]  # Morning, Evening shift type

WORKING_DAYS_EXCLUDING_CLOSED = [d for d in WORKING_DAYS if day_off[d] != 1]
STAFFS_WORKING_DAYS_SHIFTS = [(staff, day, shift) for staff in STAFFS for day in WORKING_DAYS for shift in SHIFT_TYPE]
S_FULLTIME_OR_NOT = {row.staff_name: 1 if row.fulltime == 1 else 0 for row in staff_df.itertuples()}
S_LEVEL = {row.staff_name: row.level for row in staff_df.itertuples()}

In [9]:
model = Model(sense=MINIMIZE, solver_name=CBC)
x = {(s, d, shift): model.add_var(var_type=BINARY) for s in STAFFS for d in WORKING_DAYS for shift in SHIFT_TYPE}

# One staff member on each shift for each work day
for d in WORKING_DAYS_EXCLUDING_CLOSED:
    for shift in SHIFT_TYPE:
        model += xsum(x[s, d, shift] for s in STAFFS) == 1

# Each staff member is responsible for only one shift per day
for s in STAFFS:
    for d in WORKING_DAYS_EXCLUDING_CLOSED:
        model += xsum(x[s, d, shift] for shift in SHIFT_TYPE) <= 1

# Each staff member work no more than 5 days per week
weeks = [list(WORKING_DAYS)[i : i + 7] for i in range(0, len(WORKING_DAYS), 7)]
for s in STAFFS:
    for week in weeks:
        model += xsum(x[s, d, shift] for d in week for shift in SHIFT_TYPE if d in WORKING_DAYS_EXCLUDING_CLOSED) <= 5


# There be at least one full-time employee on each workday
# for d in WORKING_DAYS_EXCLUDING_CLOSED:
#     for shift in SHIFT_TYPE:
#         model += xsum(x[s, d, shift] * S_FULLTIME_OR_NOT[s] for s in STAFFS) >= 1


# the difference in the number of working days between full-time employees is within 1 day
fulltime_staffs = [s for s in STAFFS if S_FULLTIME_OR_NOT[s] == 1]
for i in range(len(fulltime_staffs)):
    for j in range(i + 1, len(fulltime_staffs)):
        staff_i = fulltime_staffs[i]
        staff_j = fulltime_staffs[j]
        model += xsum(x[staff_i, d, shift] for d in WORKING_DAYS_EXCLUDING_CLOSED for shift in SHIFT_TYPE) - xsum(x[staff_j, d, shift] for d in WORKING_DAYS_EXCLUDING_CLOSED for shift in SHIFT_TYPE) <= 1
        model += xsum(x[staff_j, d, shift] for d in WORKING_DAYS_EXCLUDING_CLOSED for shift in SHIFT_TYPE) - xsum(x[staff_i, d, shift] for d in WORKING_DAYS_EXCLUDING_CLOSED for shift in SHIFT_TYPE) <= 1

# The difference in the number of working days between part-time employees is within 1 day
parttime_staffs = [s for s in STAFFS if S_FULLTIME_OR_NOT[s] == 0]
for i in range(len(parttime_staffs)):
    for j in range(i + 1, len(parttime_staffs)):
        staff_i = parttime_staffs[i]
        staff_j = parttime_staffs[j]
        model += xsum(x[staff_i, d, shift] for d in WORKING_DAYS_EXCLUDING_CLOSED for shift in SHIFT_TYPE) - xsum(x[staff_j, d, shift] for d in WORKING_DAYS_EXCLUDING_CLOSED for shift in SHIFT_TYPE) <= 1
        model += xsum(x[staff_j, d, shift] for d in WORKING_DAYS_EXCLUDING_CLOSED for shift in SHIFT_TYPE) - xsum(x[staff_i, d, shift] for d in WORKING_DAYS_EXCLUDING_CLOSED for shift in SHIFT_TYPE) <= 1
# Each staff member work at least one shift
for s in STAFFS:
    model += xsum(x[s, d, shift] for d in WORKING_DAYS_EXCLUDING_CLOSED for shift in SHIFT_TYPE) >= 1


# There is not much difference in the level of staff working on each date.
# min_value = min(S_LEVEL.values())
# max_value = max(S_LEVEL.values())
# for d in WORKING_DAYS_EXCLUDING_CLOSED:
#     for shift in SHIFT_TYPE:
#         level_sum = xsum(x[s, d, shift] * S_LEVEL[s] for s in STAFFS)
#         model += level_sum >= min_value + max_value  # Total Minimum Level
#         model += level_sum <= max_value + max_value  # Total Maximum Level
# Add constraint: the difference in the number of M and E shifts for each staff is at most 1
for s in STAFFS:
    model += xsum(x[s, d, "M"] for d in WORKING_DAYS_EXCLUDING_CLOSED) - xsum(x[s, d, "E"] for d in WORKING_DAYS_EXCLUDING_CLOSED) <= 1
    model += xsum(x[s, d, "E"] for d in WORKING_DAYS_EXCLUDING_CLOSED) - xsum(x[s, d, "M"] for d in WORKING_DAYS_EXCLUDING_CLOSED) <= 1


# PREVENT staff who work the “Evening: E” shift from working the “Morning: M” shift the next day.
for d in WORKING_DAYS_EXCLUDING_CLOSED:
    next_day = d + 1
    if next_day in WORKING_DAYS_EXCLUDING_CLOSED:
        for s in STAFFS:
            model += x[s, d, "E"] + x[s, next_day, "M"] <= 1

# # Constraint day-off requests
display(S_vacation_request)
for staff, leave_dates in S_vacation_request.items():
    for leave_date in leave_dates:
        if leave_date in all_days.values():
            day = [d for d in WORKING_DAYS if all_days[d] == leave_date][0]
            for shift in SHIFT_TYPE:
                model += x[staff, day, shift] == 0

model.optimize()
print(f"status: {model.status}")

{'staff_a': [datetime.date(2024, 9, 3), datetime.date(2024, 8, 30)],
 'staff_c': [datetime.date(2024, 9, 7)]}

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 154 (-14) rows, 114 (-26) columns and 1124 (-76) elements
Clp1000I sum of infeasibilities 4.34622e-11 - average 2.82222e-13, 1 fixed columns
Coin0506I Presolve 153 (-1) rows, 113 (-1) columns and 1115 (-9) elements
Clp0029I End of values pass after 113 iterations
Clp0000I Optimal - objective value 0
Clp0000I Optimal - objective value 0
Coin0511I After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Clp0000I Optimal - objective value 0
Clp0000I Optimal - objective value 0
Clp0000I Optimal - objective value 0
Coin0511I After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 0 - 0 iterations time 0.012, Presolve 0.00, Idiot 0.01

Starting MIP optimization
Cgl0002I 6 variables fixed
Cgl0004I processed model has 154 rows, 114 columns (114 integer (114 of which binary)) and 1124 elements
Coin3009W Conflict graph built in 0.000 

#### output result


In [10]:
data = [[] for _ in range(len(STAFFS))]

for i, staff in enumerate(STAFFS):
    for d in WORKING_DAYS:
        shift_temp = []
        for shift in SHIFT_TYPE:
            if x[staff, d, shift].x >= 0.99:
                shift_temp.append(shift)
        if not shift_temp:
            data[i].append("×")
        else:
            data[i].append(", ".join(shift_temp))

df = pd.DataFrame(data, index=STAFFS, columns=[all_days[d] for d in WORKING_DAYS])
display(df)

,2024-08-26,2024-08-27,2024-08-28,2024-08-29,2024-08-30,2024-08-31,2024-09-01,2024-09-02,2024-09-03,2024-09-04,2024-09-05,2024-09-06,2024-09-07,2024-09-08
staff_a,×,M,×,×,×,M,×,E,×,E,×,×,×,M
staff_b,×,×,×,×,M,E,E,×,M,×,×,E,×,×
staff_c,M,E,×,×,E,×,×,×,×,M,×,×,×,E
staff_d,×,×,M,×,×,×,M,×,E,×,×,×,E,×
staff_e,E,×,E,×,×,×,×,M,×,×,×,M,M,×


## Mixed shift type

- weekday: 2 shift type
- weekend: 3 shift type


In [11]:
STAFFS = staff_df["staff_name"].unique()
WORKING_DAYS = {day for day in range(len(all_days))}
WORKING_DAYS_EXCLUDING_CLOSED = [d for d in WORKING_DAYS if day_off[d] != 1]

WEEKDAYS = [d for d in WORKING_DAYS if all_days[d].weekday() < 5]  # weekday
WEEKENDS = [d for d in WORKING_DAYS if all_days[d].weekday() >= 5]  # weekend

SHIFT_TYPE_WEEKDAY = ["M", "E"]  # Moning, Evening
SHIFT_TYPE_WEEKEND = ["M", "E", "N"]  # Moning, Evening, Night

STAFFS_WORKING_DAYS_SHIFTS = [(staff, day, shift) for staff in STAFFS for day in WORKING_DAYS for shift in (SHIFT_TYPE_WEEKDAY if day in WEEKDAYS else SHIFT_TYPE_WEEKEND)]

S_FULLTIME_OR_NOT = {row.staff_name: 1 if row.fulltime == 1 else 0 for row in staff_df.itertuples()}
S_LEVEL = {row.staff_name: row.level for row in staff_df.itertuples()}

In [12]:
def get_shift_types(day):
    return SHIFT_TYPE_WEEKDAY if day in WEEKDAYS else SHIFT_TYPE_WEEKEND

In [13]:
model = Model(sense=MINIMIZE, solver_name=CBC)
x = {(s, d, shift): model.add_var(var_type=BINARY) for s in STAFFS for d in WORKING_DAYS for shift in SHIFT_TYPE_WEEKDAY + SHIFT_TYPE_WEEKEND}

# One staff member on each shift for each work day
for d in WORKING_DAYS_EXCLUDING_CLOSED:
    for shift in get_shift_types(d):
        model += xsum(x[s, d, shift] for s in STAFFS) == 1

# Each staff member is responsible for only one shift per day.
for s in STAFFS:
    for d in WORKING_DAYS_EXCLUDING_CLOSED:
        model += xsum(x[s, d, shift] for shift in get_shift_types(d)) <= 1

# Each staff member work no more than 5 days per week.
weeks = [list(WORKING_DAYS)[i : i + 7] for i in range(0, len(WORKING_DAYS), 7)]
for s in STAFFS:
    for week in weeks:
        model += xsum(x[s, d, shift] for d in week for shift in get_shift_types(d) if d in WORKING_DAYS_EXCLUDING_CLOSED) <= 5

# There be at least one full-time employee on each workday.
# for d in WORKING_DAYS_EXCLUDING_CLOSED:
#     for shift in get_shift_types(d):
#         model += xsum(x[s, d, shift] * S_FULLTIME_OR_NOT[s] for s in STAFFS) >= 1

# the difference in the number of working days between full-time employees is within 1 day
fulltime_staffs = [s for s in STAFFS if S_FULLTIME_OR_NOT[s] == 1]
for i in range(len(fulltime_staffs)):
    for j in range(i + 1, len(fulltime_staffs)):
        staff_i = fulltime_staffs[i]
        staff_j = fulltime_staffs[j]
        model += xsum(x[staff_i, d, shift] for d in WORKING_DAYS_EXCLUDING_CLOSED for shift in get_shift_types(d)) - xsum(x[staff_j, d, shift] for d in WORKING_DAYS_EXCLUDING_CLOSED for shift in get_shift_types(d)) <= 1
        model += xsum(x[staff_j, d, shift] for d in WORKING_DAYS_EXCLUDING_CLOSED for shift in get_shift_types(d)) - xsum(x[staff_i, d, shift] for d in WORKING_DAYS_EXCLUDING_CLOSED for shift in get_shift_types(d)) <= 1

# the difference in the number of working days between part-time employees is within 1 day
parttime_staffs = [s for s in STAFFS if S_FULLTIME_OR_NOT[s] == 0]
for i in range(len(parttime_staffs)):
    for j in range(i + 1, len(parttime_staffs)):
        staff_i = parttime_staffs[i]
        staff_j = parttime_staffs[j]
        model += xsum(x[staff_i, d, shift] for d in WORKING_DAYS_EXCLUDING_CLOSED for shift in get_shift_types(d)) - xsum(x[staff_j, d, shift] for d in WORKING_DAYS_EXCLUDING_CLOSED for shift in get_shift_types(d)) <= 1
        model += xsum(x[staff_j, d, shift] for d in WORKING_DAYS_EXCLUDING_CLOSED for shift in get_shift_types(d)) - xsum(x[staff_i, d, shift] for d in WORKING_DAYS_EXCLUDING_CLOSED for shift in get_shift_types(d)) <= 1

# ach staff member work at least one shift
for s in STAFFS:
    model += xsum(x[s, d, shift] for d in WORKING_DAYS_EXCLUDING_CLOSED for shift in get_shift_types(d)) >= 1

# There is not much difference in the level of staff working on each date
# min_value = min(S_LEVEL.values())
# max_value = max(S_LEVEL.values())
# for d in WORKING_DAYS_EXCLUDING_CLOSED:
#     for shift in get_shift_types(d):
#         level_sum = xsum(x[s, d, shift] * S_LEVEL[s] for s in STAFFS)
#         model += level_sum >= min_value + max_value  # Total Minimum Level
#         model += level_sum <= max_value + max_value  # Total Maximum Level

# Add constraint: the difference in the number of M, E and N shifts for each staff is at most 1
for s in STAFFS:
    if "M" in SHIFT_TYPE and "E" in SHIFT_TYPE:
        model += xsum(x[s, d, "M"] for d in WORKING_DAYS_EXCLUDING_CLOSED if "M" in get_shift_types(d)) - xsum(x[s, d, "E"] for d in WORKING_DAYS_EXCLUDING_CLOSED if "E" in get_shift_types(d)) <= 1
        model += xsum(x[s, d, "E"] for d in WORKING_DAYS_EXCLUDING_CLOSED if "E" in get_shift_types(d)) - xsum(x[s, d, "M"] for d in WORKING_DAYS_EXCLUDING_CLOSED if "M" in get_shift_types(d)) <= 1
    if "M" in SHIFT_TYPE and "N" in SHIFT_TYPE:
        model += xsum(x[s, d, "M"] for d in WORKING_DAYS_EXCLUDING_CLOSED if "M" in get_shift_types(d)) - xsum(x[s, d, "N"] for d in WORKING_DAYS_EXCLUDING_CLOSED if "N" in get_shift_types(d)) <= 1
        model += xsum(x[s, d, "N"] for d in WORKING_DAYS_EXCLUDING_CLOSED if "N" in get_shift_types(d)) - xsum(x[s, d, "M"] for d in WORKING_DAYS_EXCLUDING_CLOSED if "M" in get_shift_types(d)) <= 1
    if "E" in SHIFT_TYPE and "N" in SHIFT_TYPE:
        model += xsum(x[s, d, "E"] for d in WORKING_DAYS_EXCLUDING_CLOSED if "E" in get_shift_types(d)) - xsum(x[s, d, "N"] for d in WORKING_DAYS_EXCLUDING_CLOSED if "N" in get_shift_types(d)) <= 1
        model += xsum(x[s, d, "N"] for d in WORKING_DAYS_EXCLUDING_CLOSED if "N" in get_shift_types(d)) - xsum(x[s, d, "E"] for d in WORKING_DAYS_EXCLUDING_CLOSED if "E" in get_shift_types(d)) <= 1


# PREVENT staff who work the “Night: N” shift from working the “Morning: M” shift the next day.
for d in WORKING_DAYS_EXCLUDING_CLOSED:
    next_day = d + 1
    if next_day in WORKING_DAYS_EXCLUDING_CLOSED:
        for s in STAFFS:
            if "N" in get_shift_types(d) and "M" in get_shift_types(next_day):
                model += x[s, d, "N"] + x[s, next_day, "M"] <= 1

# Constrain day-off requests
for staff, leave_dates in S_vacation_request.items():
    for leave_date in leave_dates:
        if leave_date in all_days.values():
            day = [d for d in WORKING_DAYS if all_days[d] == leave_date][0]
            for shift in get_shift_types(day):
                model += x[staff, day, shift] == 0

model.optimize()
print(f"status: {model.status}")

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 132 (-11) rows, 133 (-217) columns and 1208 (-77) elements
Clp1000I sum of infeasibilities 4.65344e-11 - average 3.52534e-13, 0 fixed columns
Coin0506I Presolve 132 (0) rows, 133 (0) columns and 1208 (0) elements
Clp0029I End of values pass after 133 iterations
Clp0000I Optimal - objective value 0
Clp0000I Optimal - objective value 0
Clp0000I Optimal - objective value 0
Coin0511I After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 0 - 0 iterations time 0.012, Presolve 0.00, Idiot 0.01

Starting MIP optimization
Cgl0002I 7 variables fixed
Cgl0004I processed model has 132 rows, 133 columns (133 integer (133 of which binary)) and 1208 elements
Coin3009W Conflict graph built in 0.000 seconds, density: 1.391%
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc0045I No integer variables out of 133 objects (133 integer) have

#### output result


In [14]:
data = [[] for _ in range(len(STAFFS))]
for i, staff in enumerate(STAFFS):
    for d in WORKING_DAYS:
        shift_temp = []
        shift_types = SHIFT_TYPE_WEEKDAY if d in WEEKDAYS else SHIFT_TYPE_WEEKEND
        for shift in shift_types:
            if x[staff, d, shift].x >= 0.99:
                shift_temp.append(shift)
        if not shift_temp:
            data[i].append("×")
        else:
            data[i].append(", ".join(shift_temp))


df = pd.DataFrame(data, index=STAFFS, columns=[all_days[d] for d in WORKING_DAYS])
display(df)

,2024-08-26,2024-08-27,2024-08-28,2024-08-29,2024-08-30,2024-08-31,2024-09-01,2024-09-02,2024-09-03,2024-09-04,2024-09-05,2024-09-06,2024-09-07,2024-09-08
staff_a,×,M,×,×,×,E,M,×,×,×,×,E,×,N
staff_b,×,E,×,×,×,M,N,E,×,×,×,M,M,×
staff_c,E,×,×,×,×,×,E,×,E,M,×,×,×,M
staff_d,×,×,M,×,M,×,×,M,×,E,×,×,E,E
staff_e,M,×,E,×,E,N,×,×,M,×,×,×,N,×


### Output data (excel):

if you need


In [15]:
s = start_date.strftime("%Y_%m_%d")
e = end_date.strftime("%Y_%m_%d")
fname = "staff_schedule_" + s + "to" + e + ".xlsx"

with pd.ExcelWriter(fname, engine="xlsxwriter") as writer:
    df.to_excel(writer, sheet_name=s + "to" + e)

    workbook = writer.book
    worksheet = writer.sheets[s + "to" + e]

    date_format = workbook.add_format({"num_format": "yyyy-mm-dd"})

    worksheet.set_column(1, len(df.columns), None, date_format)

print(f"Saved to [ {fname} ]")

Saved to [ staff_schedule_2024_08_26to2024_09_08.xlsx ]
